In [11]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset = 'train', shuffle=True)

In [12]:
print(twenty_train.target_names) #prints all the categories
print("\n".join(twenty_train.data[0].split("\n")[:3])) #prints first line of the first data file

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


In [27]:
print(twenty_train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [29]:
print(twenty_train.target[0])
print(twenty_train.target_names[twenty_train.target[0]])

7
rec.autos


In [14]:
#data = open('train.csv','r')

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

In [17]:
from sklearn.naive_bayes import MultinomialNB
#clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [18]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [19]:
import numpy as np
twenty_test = fetch_20newsgroups(subset = 'test', shuffle = True)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.81691449814126393

In [20]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect',CountVectorizer(stop_words = 'english')),
                        ('tfidf',TfidfTransformer()),
                        ('clf-svm',SGDClassifier(loss = 'hinge', penalty = 'l2',
                                                 alpha = 1e-3, n_iter = 5,
                                                 random_state = 42)),
                        ])
text_clf_svm = text_clf_svm.fit(twenty_train.data,twenty_train.target)
predicted_svm = text_clf_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)

0.82249070631970256

In [21]:
'''
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range':[(1,1),(1,2)],
             'tfidf__use_idf':(True,False),
             'clf__alpha': (1e-2, 1e-3),
             }
'''

"\nfrom sklearn.model_selection import GridSearchCV\nparameters = {'vect__ngram_range':[(1,1),(1,2)],\n             'tfidf__use_idf':(True,False),\n             'clf__alpha': (1e-2, 1e-3),\n             }\n"

In [22]:
#gs_clf = GridSearchCV(text_clf, parameters, n_jobs = -1)
#gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

In [23]:
#gs_clf.best_score_
#gs_clf.best_params_

In [24]:
'''
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                   'tfidf__use_idf': (True, False),
                   'clf-svm__alpha': (1e-2, 1e-3),
 }
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_
'''
#gs_clf_svm.best_params_

"\nparameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],\n                   'tfidf__use_idf': (True, False),\n                   'clf-svm__alpha': (1e-2, 1e-3),\n }\ngs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)\ngs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)\ngs_clf_svm.best_score_\n"

In [25]:
#import nltk
#nltk.download()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                             ('tfidf', TfidfTransformer()),
                             ('mnb', MultinomialNB(fit_prior=False)),
 ])

text_mnb_stemmed = text_mnb_stemmed.fit(twenty_train.data, twenty_train.target)

predicted_mnb_stemmed = text_mnb_stemmed.predict(twenty_test.data)

np.mean(predicted_mnb_stemmed == twenty_test.target)

0.81678173127987252

In [26]:
text_svm_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('svm', SGDClassifier(loss = 'hinge', penalty = 'l2',
                                                 alpha = 1e-3, n_iter = 5,
                                                 random_state = 42)),
 ])

text_svm_stemmed = text_svm_stemmed.fit(twenty_train.data, twenty_train.target)

predicted_svm_stemmed = text_svm_stemmed.predict(twenty_test.data)

np.mean(predicted_svm_stemmed == twenty_test.target)

0.81943706850770048